In [65]:
import numpy as np
import pandas as pd
from selenium import webdriver
from requests_html import HTML
# -*- coding: UTF-8 -*-
from bs4 import BeautifulSoup
import re
import time

In [18]:
data = pd.read_csv('0-25k.csv')
print('Data Shape: ', data.shape)
data.head(5)

Data Shape:  (25000, 4)


,price,type,time,link
0,18.000.000.000 đ,Môi giới,2019-12-04 14:48:18.407093,https://nha.chotot.com//tp-ho-chi-minh/quan-ta...
1,1.750.000.000 đ,Môi giới,2019-12-04 14:48:18.408963,https://nha.chotot.com//tp-ho-chi-minh/huyen-b...
2,1.250.000.000 đ,Đại Phong,2019-12-04 14:48:18.411019,https://nha.chotot.com//tp-ho-chi-minh/huyen-b...
3,2.000.000.000 đ,Môi giới,2019-12-04 14:48:18.412744,https://nha.chotot.com//tp-ho-chi-minh/quan-th...
4,7.400.000.000 đ,Nhã Phương,2019-12-04 14:48:18.414617,https://nha.chotot.com//tp-ho-chi-minh/quan-7/...


### After test for javascript, we saw that site use javascript to render page, so we decide to use selenium.webdriver to crawl information from attribute link.

1. Số phòng ngủ
2. Diện tích
3. Loại hình nhà ở
4. Tiêu đề
5. Mô tả chi tiết
6. Số phòng vệ sinh
7. Hướng cửa chính
8. Giấy tờ pháp lý
9. Tên phân khu/lô/block/tháp
10. Số hiệu
11. Tổng số tầng
12. Đặc điểm nhà đất
13. Địa chỉ

Có những cái như `Tiêu đề, Mô tả chi tiết, Địa chỉ` ta có thể lấy riêng được, nên tạo một dict chứa những thuộc tính còn lại

In [19]:
def init_dict():
    return {}

In [20]:
def get_description(r):
    p_list = r.find('p')
    descrip = [i for i in p_list if 'itemprop' in i.attrs and i.attrs['itemprop'] == 'description'][0]
    descrip = BeautifulSoup(descrip.html, 'html.parser')
    strgs = []
    for string in descrip.strings:
        strg = re.sub('\\xa0','', string)
        strg = re.sub('^ +','', strg)
        strg = re.sub(' +$', '', strg)
        strgs.append(strg)
    return '\n'.join(strgs)
# Test
# get_description(r)

In [52]:
link = data.link[15823:20000]
link

15823    https://nha.chotot.com//tp-ho-chi-minh/quan-bi...
15824    https://nha.chotot.com//tp-ho-chi-minh/quan-8/...
15825    https://nha.chotot.com//tp-ho-chi-minh/quan-go...
15826    https://nha.chotot.com//tp-ho-chi-minh/quan-10...
15827    https://nha.chotot.com//tp-ho-chi-minh/quan-go...
                               ...                        
19995    https://nha.chotot.com//tp-ho-chi-minh/quan-ph...
19996    https://nha.chotot.com//tp-ho-chi-minh/quan-bi...
19997    https://nha.chotot.com//tp-ho-chi-minh/huyen-h...
19998    https://nha.chotot.com//tp-ho-chi-minh/quan-9/...
19999    https://nha.chotot.com//tp-ho-chi-minh/quan-12...
Name: link, Length: 4177, dtype: object

In [66]:
link_list = []
tieu_de_list = []
mtct_list = []
diachi_list = []
dict_list = []
cnt = 0
ff = webdriver.Firefox()


for i in range(len(link)):
    
    links = link.values[i]
    dicts = init_dict()
    ff.get(links)
    try:
        ff.find_element_by_css_selector('.ctTooltipClose').click()
    except:
        i = 1
        
    time.sleep(1)
    r = HTML(html = ff.page_source.encode('utf-8'))

    tmp = r.find('h1', first=True)
    if tmp == None:
        continue
    tit = tmp.text
    desp = get_description(r)
    tmp = r.find('.media-body.media-middle')[0]
    addr = tmp.text
    
    l = r.find('._1DTCXk4eb6dusdylsQxCos', first=True)
    for i in l.find('span'):
        tmpd = i.text.split(': ')
        if len(tmpd) < 2:
            continue
        pre = tmpd[0].lower()
        print(repr(tmpd))
        dicts[pre] = tmpd[1]

    link_list.append(links)
    tieu_de_list.append(tit)
    mtct_list.append(desp)
    diachi_list.append(addr)
    dict_list.append(dicts)
    
ff.close()

['Diá»\x87n tÃ\xadch', '85200 mÂ²']
['Sá»\x91 phÃ²ng ngá»§', '5 phÃ²ng']
['Loáº¡i hÃ¬nh nhÃ\xa0 á»\x9f', 'NhÃ']
['Giáº¥y tá»\x9d phÃ¡p lÃ½', 'Ä\x90Ã£ cÃ³ sá»\x95 Ä\x91á»\x8f/sá»\x95 há»\x93ng']


InvalidArgumentException: Message: unexpected end of hex escape at line 1 column 44373


In [54]:
len(link_list)

1

In [49]:
df = pd.DataFrame({'link':link_list, 'title':tieu_de_list, 'describe':mtct_list, 'address':diachi_list, 'dict': dict_list})
df.to_csv('15823-20000.csv',index=None)

In [ ]:
link = data.link[20001:25000]
link

In [ ]:
link_list = []
tieu_de_list = []
mtct_list = []
diachi_list = []
dict_list = []
cnt = 0
ff = webdriver.Firefox()


for i in range(len(link)):
    
    links = link.values[i]
    dicts = init_dict()
    ff.get(links)
    try:
        ff.find_element_by_css_selector('.ctTooltipClose').click()
    except:
        i = 1
        
    time.sleep(1)
    r = HTML(html = ff.page_source)

    tmp = r.find('h1', first=True)
    if tmp == None:
        continue
    tit = tmp.text
    desp = get_description(r)
    tmp = r.find('.media-body.media-middle')[0]
    addr = tmp.text
    
    l = r.find('._1DTCXk4eb6dusdylsQxCos', first=True)
    for i in l.find('span'):
        tmpd = i.text.split(': ')
        if len(tmpd) < 2:
            continue
        pre = tmpd[0].lower()
        dicts[pre] = tmpd[1]

    link_list.append(links)
    tieu_de_list.append(tit)
    mtct_list.append(desp)
    diachi_list.append(addr)
    dict_list.append(dicts)
    
ff.close()

In [ ]:
df = pd.DataFrame({'link':link_list, 'title':tieu_de_list, 'describe':mtct_list, 'address':diachi_list, 'dict': dict_list})
df.to_csv('20001-25000.csv',index=None)

In [ ]:
link = data.link[10001:15000]
link

In [ ]:
link_list = []
tieu_de_list = []
mtct_list = []
diachi_list = []
dict_list = []
cnt = 0
ff = webdriver.Firefox()


for i in range(len(link)):
    
    links = link.values[i]
    dicts = init_dict()
    ff.get(links)
    try:
        ff.find_element_by_css_selector('.ctTooltipClose').click()
    except:
        i = 1
        
    time.sleep(1)
    r = HTML(html = ff.page_source)

    tmp = r.find('h1', first=True)
    if tmp == None:
        continue
    tit = tmp.text
    desp = get_description(r)
    tmp = r.find('.media-body.media-middle')[0]
    addr = tmp.text
    
    l = r.find('._1DTCXk4eb6dusdylsQxCos', first=True)
    for i in l.find('span'):
        tmpd = i.text.split(': ')
        pre = tmpd[0].lower()
        dicts[pre] = tmpd[1]

    link_list.append(links)
    tieu_de_list.append(tit)
    mtct_list.append(desp)
    diachi_list.append(addr)
    dict_list.append(dicts)
    
ff.close()

In [ ]:
df = pd.DataFrame({'link':link_list, 'title':tieu_de_list, 'describe':mtct_list, 'address':diachi_list, 'dict': dict_list})
df.to_csv('10001-15000.csv',index=None)